In [ ]:
# import torch

# # Define test path
# model_path = 'data/04_model/eden/model/best_model.pt' # Adjust this path as needed

# # Load the state dictionary
# try:
#     state_dict = torch.load(model_path)
#     print("Model state dictionary loaded successfully!")
# except Exception as e:
#     print(f"Error loading model: {e}")
#     exit() # Exit if loading fails

# # --- Inspect the contents ---

# # Print all keys (layer names) in the state dictionary
# print("\nKeys in the state dictionary:")
# for key in state_dict.keys():
#     print(key)

# # Inspect the shape and device of a few params
# print("\nExample parameters from the state dictionary (first few keys):\n")Ÿ
# for i, (key, value) in enumerate(state_dict.items()):
#     if i >= 5: # Limit to first 5
#         break
#     print(f"  Key: {key}")
#     print(f"  Shape: {value.shape}")
#     print(f"  Device: {value.device}")
#     print(f"      Value (first 5 elements): {value.flatten()[:5].tolist()}\n")

# if 'gat_layers.0.lin_src.weight' in state_dict:
#     print(f"\nShape of first GAT layer weights: {state_dict['gat_layers.0.lin_src.weight'].shape}")

In [ ]:
# Load library imports
import os
import sys
import torch
import random
import logging
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
from collections import Counter
from permetrics.regression import RegressionMetric

# Load project Imports
from src.utils.config_loader import load_project_config, deep_format, expanduser_tree
from src.model.model_building import build_data_loader, instantiate_model_and_associated
from src.utils.config_loader import load_project_config
from src.graph_building.graph_construction import build_mesh, \
    define_catchment_polygon, define_graph_adjacency
from src.preprocessing.data_partitioning import define_station_id_splits, \
    load_graph_tensors, build_pyg_object
from src.preprocessing.model_feature_engineering import preprocess_gwl_features, \
    preprocess_shared_features

In [ ]:
# Set up logger config
logging.basicConfig(
    level=logging.INFO,
   format='%(levelname)s - %(message)s',
#    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

# Set up logger for file and load config file for paths and params
logger = logging.getLogger(__name__)
config = load_project_config(config_path="config/project_config.yaml")
notebook = True

# Set up root directory paths in config
raw_data_root = config["global"]["paths"]["raw_data_root"]
results_root = config["global"]["paths"]["results_root"]

# Reformat config roots
config = deep_format(
    config,
    raw_data_root=raw_data_root,
    results_root=results_root
)
config = expanduser_tree(config)

In [ ]:
# Set up seeding to define global states
random_seed = config["global"]["pipeline_settings"]["random_seed"]
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define notebook demo catchment
catchments_to_process = config["global"]["pipeline_settings"]["catchments_to_process"]
catchment = catchments_to_process[0]
run_defra_API_calls = config["global"]["pipeline_settings"]["run_defra_api"]

logger.info(f"Show Notebook Outputs: {notebook}")
logger.info(f"Notebook Demo Catchment: {catchment.capitalize()}")

In [ ]:
# # Select Catchment area from country wide gdf
# define_catchment_polygon(
#     england_catchment_gdf_path=config[catchment]['paths']['gis_catchment_boundary'],
#     target_mncat=config[catchment]['target_mncat'],
#     catchment=catchment,
#     polygon_output_path=config[catchment]['paths']['gis_catchment_dir']
# )

# # Build catchment mesh
# mesh_nodes_table, mesh_nodes_gdf, mesh_cells_gdf_polygons, catchment_polygon = build_mesh(
#     shape_filepath=config[catchment]['paths']['gis_catchment_dir'],
#     output_path=config[catchment]['paths']['mesh_nodes_output'],
#     catchment=catchment,
#     grid_resolution=config[catchment]['preprocessing']['graph_construction']['grid_resolution']
# )

# logger.info(f"Pipeline step 'Build Mesh' complete for {catchment} catchment.")

In [ ]:
# directional_edge_path = config[catchment]["paths"]["direction_edge_weights_path"]
# directional_edge_weights = pd.read_csv(directional_edge_path)

# # Create specific node_id column to merge
# directional_edge_weights["node_id"] = range(0, len(directional_edge_weights))
# directional_edge_weights

In [ ]:
# # Load in directional edge weights and mean elevation (not req. in main pipeline)
# directional_edge_path=config[catchment]["paths"]["direction_edge_weights_path"]
# directional_edge_weights = pd.read_csv(directional_edge_path)

# edge_attr_tensor, edge_index_tensor = define_graph_adjacency(
#     directional_edge_weights=directional_edge_weights,
#     elevation_geojson_path=config[catchment]['paths']['elevation_geojson_path'],
#     graph_output_dir=config[catchment]["paths"]["graph_data_output_dir"],
#     mesh_cells_gdf_polygons=mesh_cells_gdf_polygons,
#     epsilon_path=config["global"]["graph"]["epsilon"],
#     catchment=catchment
# )

# logger.info(f"Pipeline step 'Define Graph Adjacency' complete for {catchment} catchment.\n")

In [ ]:
# # Load tensors from file if needed
# edge_index_tensor, edge_attr_tensor = load_graph_tensors(
#     graph_output_dir=config[catchment]["paths"]["graph_data_output_dir"],
#     catchment=catchment
# )

# # Load main_df_full from file if needed
# load_path = config[catchment]["paths"]["final_df_path"] + 'final_df.csv'
# main_df_full = pd.read_csv(load_path)
# main_df_full

In [ ]:
# # --- 6a. Define Spatial Split for Observed Stations ---

# train_station_ids, val_station_ids, test_station_ids = define_station_id_splits(
#     main_df_full=main_df_full,
#     catchment=catchment,
#     test_station_shortlist=config[catchment]["model"]["data_partioning"]["test_station_shortlist"],
#     val_station_shortlist=config[catchment]["model"]["data_partioning"]["val_station_shortlist"],
#     random_seed=config["global"]["pipeline_settings"]["random_seed"],
#     output_dir=config[catchment]["paths"]["aux_dir"],
#     perc_train=config[catchment]["model"]["data_partioning"]["percentage_train"],
#     perc_val=config[catchment]["model"]["data_partioning"]["percentage_val"],
#     perc_test=config[catchment]["model"]["data_partioning"]["percentage_test"]
# )

# logger.info(f"Pipeline Step 'define station splits' complete for {catchment} catchment.")

In [ ]:
# # --- 6b. Preprocess (Standardise, one hot encode, round to 4dp) all shared features (not GWL) ---

# processed_df, shared_scaler, shared_encoder, gwl_feats = preprocess_shared_features(
#     main_df_full=main_df_full,
#     catchment=catchment,
#     random_seed=config["global"]["pipeline_settings"]["random_seed"],
#     violin_plt_path=config[catchment]["visualisations"]["violin_plt_path"],
#     scaler_dir = config[catchment]["paths"]["scalers_dir"],
#     aux_dir=config[catchment]["paths"]["aux_dir"]
# )

# logger.info(f"Pipeline Step 'Preprocess Final Shared Features' complete for {catchment} catchment.")

In [ ]:
# # --- 6c. Preprocess all GWL features using training data only ---

# processed_df, gwl_scaler, gwl_encoder = preprocess_gwl_features(
#     processed_df=processed_df,
#     catchment=catchment,
#     train_station_ids=train_station_ids,
#     val_station_ids=val_station_ids,
#     test_station_ids=test_station_ids,
#     sentinel_value = config["global"]["graph"]["sentinel_value"],
#     scaler_dir = config[catchment]["paths"]["scalers_dir"]
# )

# logger.info(f"Pipeline Step 'Preprocess Final GWL Features' complete for {catchment} catchment.")

In [ ]:
# processed_df_test = processed_df.drop(columns=['streamflow_total_m3', 'HOST_soil_class_freely_draining_soils', 'HOST_soil_class_high_runoff_(impermeable)', 
#                                                'HOST_soil_class_impeded_saturated_subsurface_flow', 'HOST_soil_class_peat_soils', 'aquifer_productivity_High',
#                                                'aquifer_productivity_Low', 'aquifer_productivity_Mixed', 'aquifer_productivity_Moderate',
#                                                'aquifer_productivity_nan']).copy()

In [ ]:
all_timesteps_list = torch.load(config[catchment]["paths"]["pyg_object_path"])
all_timesteps_list

In [ ]:
# --- 7a. Build Data Loaders by Timestep ---

full_dataset_loader = build_data_loader(
    all_timesteps_list=all_timesteps_list,
    batch_size = config["global"]["model"]["data_loader_batch_size"],
    shuffle = config["global"]["model"]["data_loader_shuffle"],
    catchment=catchment
)

logger.info(f"Pipeline Step 'Create PyG DataLoaders' complete for {catchment} catchment.")

In [ ]:
# --- 7b. Define Graph Neural Network Architecture ---

model, device, optimizer, criterion = instantiate_model_and_associated(
    all_timesteps_list=all_timesteps_list,
    config=config,
    catchment=catchment
)

logger.info(f"Pipeline Step 'Instantiate GAT-LSTM Model' complete for {catchment} catchment.")

In [ ]:
import torch
import joblib
import logging
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error

path = "data/04_model/eden/model/pt_model/model_20250813-193958_GATTrue_LSTMFalse_GATH12_GATD0-4_GATHC64_GATOC64_GATNL2_LSTHC32_LSTNL1_OUTD1_LR0-001_WD0-001_SM0-1_E250_ESP30_LRSF0-5_LRSP8_MINLR1e-06_LD0-0001_GCMN1-0.pt"
iteration = 1  # Complete

best_model = model  # Assume model object already defined and moved to correct device
best_model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
best_model.eval()
logger.info(f"Loaded best model from {path}")

# Load target scaler
scaler_path = "data/03_graph/eden/scalers/target_scaler.pkl"
target_scaler = joblib.load(scaler_path)
logger.info(f"Loaded target scaler from: {scaler_path}")

scale = torch.tensor(target_scaler.scale_, device=device)
mean = torch.tensor(target_scaler.mean_, device=device)

# Initialise global LSTM state
if best_model.run_LSTM:
    lstm_state_store = {
        'h': torch.zeros(best_model.num_layers_lstm, best_model.num_nodes, best_model.hidden_channels_lstm).to(device),
        'c': torch.zeros(best_model.num_layers_lstm, best_model.num_nodes, best_model.hidden_channels_lstm).to(device)
    }
else:
    lstm_state_store = None

# Prepare for evaluation
test_predictions_unscaled = []
test_actuals_unscaled = []

logger.info("--- Starting Model Evaluation on Test Set ---")
test_loop = tqdm(all_timesteps_list, desc="Evaluating on Test Set", leave=False)

with torch.no_grad():
    for i, data in enumerate(test_loop):
        data = data.to(device)
        test_mask = data.test_mask

        if test_mask.sum() == 0:
            continue

        # Model forward pass on full node set
        predictions_all, (h_new, c_new), returned_node_ids = best_model(
            x=data.x,
            edge_index=data.edge_index,
            edge_attr=data.edge_attr,
            current_timestep_node_ids=data.node_id,
            lstm_state_store=lstm_state_store
        )

        # Update LSTM memory for current nodes
        if best_model.run_LSTM:
            lstm_state_store['h'][:, returned_node_ids, :] = h_new.detach()
            lstm_state_store['c'][:, returned_node_ids, :] = c_new.detach()

        # Filter predictions/targets to test nodes
        preds_std = predictions_all[test_mask]
        targets_std = data.y[test_mask]

        # Inverse transform to original scale
        preds_np = preds_std.cpu().numpy()
        targets_np = targets_std.cpu().numpy()

        preds_unscaled = target_scaler.inverse_transform(preds_np)
        targets_unscaled = target_scaler.inverse_transform(targets_np)

        test_predictions_unscaled.extend(preds_unscaled.flatten())
        test_actuals_unscaled.extend(targets_unscaled.flatten())

        if i < 5:  # Show first few predictions
            print("Sample predictions (m AOD):", preds_unscaled[:5].flatten())
            print("Sample actuals     (m AOD):", targets_unscaled[:5].flatten())

# Final evaluation
if len(test_actuals_unscaled) > 0:
    loss_type = config[catchment]["training"]["loss"]

    if loss_type == "MAE":
        final_test_metric = mean_absolute_error(test_actuals_unscaled, test_predictions_unscaled)
        logger.info(f"\n--- Final Test Set MAE (m AOD): {final_test_metric:.4f} ---\n")

    elif loss_type == "MSE":
        final_test_metric = mean_squared_error(test_actuals_unscaled, test_predictions_unscaled)
        logger.info(f"\n--- Final Test Set MSE (m AOD²): {final_test_metric:.4f} ---\n")

    else:
        logger.warning(f"Unrecognized loss type '{loss_type}' in config — skipping final metric calculation.")
else:
    logger.warning("No test data found — check 'data.test_mask'.")

logger.info("--- Model Evaluation on Test Set Complete ---")

In [ ]:
# Load the target scaler
from joblib import load

# Load target scaler (y, 'gwl_value') in
scalers_dir = config[catchment]["paths"]["scalers_dir"]
target_scaler_path = os.path.join(scalers_dir, "target_scaler.pkl")
try:
    target_scaler = load(target_scaler_path)
    logger.info(f"Successfully loaded target scaler from: {target_scaler_path}")
except Exception as e:
    logger.warning(f"No target scaler found or error loading it: {e}")
    target_scaler = None

# Convert both to np array
test_predictions_np = np.array(test_predictions_unscaled).reshape(-1, 1)
test_actuals_np = np.array(test_actuals_unscaled).reshape(-1, 1)

# Confirm range (sanity checker)
logger.info(f"Sample prediction range: {test_predictions_np.min():.2f} to {test_predictions_np.max():.2f}")
logger.info(f"Sample actual range:     {test_actuals_np.min():.2f} to {test_actuals_np.max():.2f}")

# Assign reshaped vals to final arrs for plotting and metrics
test_predictions_final = test_predictions_np
test_actuals_final = test_actuals_np

In [ ]:
from sklearn.metrics import mean_absolute_error
from permetrics.regression import RegressionMetric

final_test_mae = mean_absolute_error(test_actuals_final, test_predictions_final)
adjusted_mae = mean_absolute_error(test_actuals_final, test_predictions_final-final_test_mae)
unit_label = "mAOD" if target_scaler else "standard units"
evaluator = RegressionMetric(test_actuals_final, test_predictions_final)  # Before offset correction
adj_evaluator = RegressionMetric(test_actuals_final, test_predictions_final-final_test_mae)  # After offset correction

# Mean Absolute Error (MAE)
# logger.info(f"Final Test Set MAE: {final_test_mae:.4f} {unit_label} [Target 0.2 to 0.5 mAOD]")
logger.info(f"Final Adjusted MAE: {adjusted_mae:.4f} {unit_label} [Target 0.2 to 0.5 mAOD]\n")

# Root Mean Square Error (RMSE)
# logger.info(f"Baseline RMSE: {evaluator.root_mean_squared_error():.4f} {unit_label} [Target 0.25 to 0.6 mAOD]")
logger.info(f"Adjusted RMSE: {adj_evaluator.root_mean_squared_error():.4f} {unit_label} [Target 0.25 to 0.6 mAOD]\n")

# Coefficient of Determination (R^2)
# logger.info(f"Baseline R^2: {evaluator.coefficient_of_determination():.4f} [Target 0.80 or higher]")
logger.info(f"Adjusted R^2: {adj_evaluator.coefficient_of_determination():.4f} [Target 0.80 or higher]\n")

# Nash-Sutcliffe Efficiency (NSE)
# logger.info(f"Baseline NSE: {evaluator.nash_sutcliffe_efficiency():.4f} [Target 0.75 or higher]")
logger.info(f"Adjusted NSE: {adj_evaluator.nash_sutcliffe_efficiency():.4f} [Target 0.75 or higher]\n")

# Kling Gupta Efficiency (KGE)
# logger.info(f"Baseline KGE: {evaluator.kling_gupta_efficiency():.4f} [Target 0.75 or higher]")
logger.info(f"Adjusted KGE: {adj_evaluator.kling_gupta_efficiency():.4f} [Target 0.75 or higher]\n")

In [ ]:
import numpy as np

def calculate_kge_components(actuals, predictions):
    """Get KGE Component parts as individual values."""
    r = np.corrcoef(actuals.flatten(), predictions.flatten())[0, 1]
    beta = np.mean(predictions.flatten()) / np.mean(actuals.flatten())
    gamma = np.std(predictions.flatten()) / np.std(actuals.flatten())
    return r, beta, gamma

# Using precalc'd adjusted predictions
adjusted_predictions = test_predictions_final - mean_absolute_error(test_actuals_final, test_predictions_final)
r_actual, beta_actual, gamma_actual = calculate_kge_components(test_actuals_final, test_predictions_final)
r_adjusted, beta_adjusted, gamma_adjusted = calculate_kge_components(test_actuals_final, adjusted_predictions)

# Log component results (baseline)
print(f"Baseline KGE Components:")
print(f"  Correlation (r): {r_actual:.4f};")
print(f"  Bias (beta): {beta_actual:.4f};")
print(f"  Variability (gamma): {gamma_actual:.4f}\n")

# Log component results (adjusted)
print(f"Adjusted KGE Components:")
print(f"  Correlation (r): {r_adjusted:.4f};")
print(f"  Bias (beta): {beta_adjusted:.4f};")
print(f"  Variability (gamma): {gamma_adjusted:.4f}\n")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 7))

# plt.plot(test_actuals_final, label='Actual GWL Value', color='blue', alpha=0.7)
plt.plot(test_predictions_final - final_test_mae, label='Predicted GWL Value', color='red', alpha=0.7)  # , linestyle='--'
# plt.plot(test_predictions_final, label='Predicted GWL Value', color='red', alpha=0.7)  # , linestyle='--'
plt.plot(test_actuals_final, label='Actual GWL Value', color='blue', alpha=0.7)

plt.title('Actual vs. Predicted Groundwater Levels on Test Set')
plt.xlabel('Data Point Index (Sequential Timesteps/Stations)')
plt.ylabel('Groundwater Level (m AOD)' if target_scaler else 'Standardised GWL')
plt.legend()
plt.grid(True)
plt.tight_layout()

# Give slightly more room above and below than automatic
all_vals = np.concatenate([test_actuals_final, test_predictions_final - final_test_mae])
y_min = all_vals.min()
y_max = all_vals.max()
y_range = y_max - y_min
plt.ylim(y_min - y_range/3, y_max + y_range/3)

# Save plot
base_name = os.path.basename(path)
filename_no_ext, extension = os.path.splitext(base_name)
save_path = "results/trained_models/eden/" + f"{iteration}_" + filename_no_ext
# save_path = os.path.join("results/trained_models/eden/2_", filename_no_ext)
plt.savefig(save_path, dpi=300)

plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set a style for plots (optional)
sns.set_style("whitegrid")

# Create a figure and a set of subplots
plt.figure(figsize=(15, 5))

# Plot the predicted values
plt.plot(test_predictions_unscaled, label='Predicted GWL Value', color='red', alpha=0.7)  # , linestyle='--'

# Add titles and labels
plt.title('Predicted Groundwater Levels on Test Set')
plt.xlabel('Data Point Index (Sequential Timesteps/Stations)')
plt.ylabel('Groundwater Level (m AOD)')
plt.legend()
plt.grid(True)
plt.tight_layout() 

plt.show()

logger.info("Pipeline step 'Generate plot of predicted values' complete.")

Plot at a weekly resolution (using daily predictions)

In [ ]:
# Weekly (7-day) mean plot from daily arrays 

pred_daily = np.asarray(test_predictions_final).reshape(-1)
act_daily = np.asarray(test_actuals_final).reshape(-1)

# Calc means
k = 7
n_full = (len(act_daily) // k) * k
if n_full < len(act_daily):
    logger.info(f"Trimming {len(act_daily) - n_full} trailing day(s).")
act_weekly  = act_daily[:n_full].reshape(-1, k).mean(axis=1)
pred_weekly = pred_daily[:n_full].reshape(-1, k).mean(axis=1)

plt.figure(figsize=(15, 7))
plt.plot(act_weekly,  label='Actual GWL', alpha=0.8)
plt.plot(pred_weekly - final_test_mae, label='Predicted GWL', alpha=0.8)  # linestyle='--',
plt.title('Actual vs. Predicted Groundwater Levels (Weekly Means)')
plt.xlabel('Week Index')
plt.ylabel('Groundwater Level (m AOD)' if target_scaler else 'Standardised GWL')
plt.legend()
plt.grid(True)
plt.tight_layout()

# Set plot limits
all_vals = np.concatenate([act_weekly, pred_weekly - final_test_mae])
y_min = all_vals.min()
y_max = all_vals.max()
y_range = y_max - y_min
plt.ylim(y_min - y_range/2, y_max + y_range/2)

# Save next to the daily figure with _weekly
weekly_save_path = save_path + "_weekly"
# plt.savefig(weekly_save_path, dpi=300)

plt.show()


In [ ]:
import pandas as pd
from ray.tune import ExperimentAnalysis
import os
import logging
import sys
import glob
import numpy as np

logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

ray_tune_dir = "data/04_model/eden/model/ray_tune_gwl"

if not os.path.isdir(ray_tune_dir):
    logger.error(f"Error: The specified directory does not exist or is not a directory: {ray_tune_dir}")
else:
    try:
        abs_dir = os.path.abspath(ray_tune_dir)
        uri = f"file://{abs_dir}"

        analysis = ExperimentAnalysis(uri)
        df = analysis.dataframe()

        # Coerce numeric metrics (in case anything logged as strings)
        for col in ['val_loss', 'train_loss', 'epoch']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # Keep only rows with a valid trial_id
        if 'trial_id' not in df.columns:
            raise KeyError("Expected 'trial_id' column not found in Ray Tune results DataFrame.")
        df = df[df['trial_id'].notna()].copy()

        # filter out errored trials (errored due to timeouts)
        if 'error' in df.columns:
            df = df[df['error'].isna()].copy()

        logger.info(f"Total rows (iterations) loaded: {len(df)}")
        n_trials = df['trial_id'].nunique()
        logger.info(f"Trials represented: {n_trials}")

        # Compute per-trial aggregates
        config_cols = [c for c in df.columns if c.startswith('config/')]
        keep_cols = ['trial_id', 'logdir'] + config_cols
        meta_first = (df[keep_cols]
                      .sort_values(['trial_id']) 
                      .groupby('trial_id', as_index=False)
                      .first())
        
        # Trials that never logged val_loss/train_los (timed out) become NaN -> drop for ranking
        agg = (df.groupby('trial_id')
                 .agg(min_val_loss=('val_loss', 'min'),
                      mean_val_loss=('val_loss', 'mean'),
                      min_train_loss=('train_loss', 'min'),
                      mean_train_loss=('train_loss', 'mean'),
                      last_epoch=('epoch', 'max'),
                      last_iter=('training_iteration', 'max'))
                 .reset_index())

        # Merge configs back in
        per_trial = meta_first.merge(agg, on='trial_id', how='left')

        # Keep only trials with at least some val_loss signal
        ranked = per_trial[per_trial['min_val_loss'].notna()].copy()
        if ranked.empty:
            raise RuntimeError("No trials have non-NaN 'val_loss'. Cannot rank.")

        ranked = ranked.sort_values(['min_val_loss', 'mean_val_loss'], ascending=[True, True]).reset_index(drop=True)

        best = ranked.iloc[0]
        trial_id = best['trial_id']

        # Pack nice dicts for logging
        best_config = {k: best[k] for k in config_cols if k in ranked.columns}
        best_metrics = {k: best[k] for k in ['min_val_loss', 'mean_val_loss', 'min_train_loss', 'mean_train_loss',
                                             'last_epoch', 'last_iter'] if k in ranked.columns}

        logger.info("\n--- Best Trial (by min_val_loss, then mean_val_loss) ---")
        logger.info(f"Trial ID: {trial_id}")
        logger.info(f"Best metrics: {best_metrics}")
        logger.info(f"Best hyperparameters: {best_config}")

        # Locate the corresponding PT model for this trial_id
        pt_root = os.path.abspath("data/04_model/eden/model/pt_model")
        pattern = os.path.join(pt_root, f"trial_{trial_id}", "pt_model", "*.pt")
        pt_candidates = sorted(glob.glob(pattern))
        if pt_candidates:
            best_pt_model_path = pt_candidates[-1]  # choose last
            logger.info(f"Best trial PT model: {best_pt_model_path}")
        else:
            logger.warning(f"No .pt model files found for trial_id={trial_id} with pattern: {pattern}")

        # Show a compact table of the top 10 trials
        display_cols = (['trial_id'] + config_cols +
                        ['min_val_loss', 'mean_val_loss', 'min_train_loss', 'mean_train_loss', 'last_epoch'])
        display_cols = [c for c in display_cols if c in ranked.columns]
        top10 = ranked[display_cols].head(10)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 160)
        print("\nTop 10 trials by validation loss:")
        print(top10.to_string(index=False))

    except Exception as e:
        logger.error(f"An error occurred during analysis: {e}")
